# Utiliser Voilà pour transformer votre Notebook Jupyter en application Web

Voici un exemple de création d'une interface interactive à l'aide de widgets IPython dans Jupyter Notebook.
- Curseur pour entrer un numéro et un bouton pour le doubler, affichant le résultat dans une zone de sortie lorsque le bouton est cliqué.

In [ ]:
#Librairies nécessaires
# !pip install ipywidgets
# !pip install plotly
# !pip install voila

from ipywidgets import Dropdown
from ipywidgets import Layout
from ipywidgets import HBox, VBox
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
from sklearn.datasets import load_iris
import plotly.graph_objects as go

In [ ]:
# Définir les widgets
slider = widgets.IntSlider(description='Nombre:')
button = widgets.Button(description='Double')
output = widgets.Output()

# Fonction permettant de doubler le nombre
def double_number(b):
    with output:
        output.clear_output()
        print("Le double de {} est:".format(slider.value), 2 * slider.value)

# Attribuer une fonction à la touche
button.on_click(double_number)

# Afficher les widgets
display(slider, button, output)


### Construire une visualisation interactive avec le Jeu de données Iris.

In [ ]:
#charger le jeu de données Iris.

data = load_iris()
df = pd.DataFrame(data = data['data'], columns = data.feature_names)
species_dict = {0:'setosa', 1: 'versicolor', 2: 'virginica'}
df['species'] = [species_dict[x] for x in data.target]
#df

In [ ]:
#Construire la visualisation qui sera affichée dans l'application web.
#on utilise une sous-classe des objets de Plotly appelée FigureWidget. 
#L'utilisation de cette classe de visualisation permettra d'ajouter
#l'interactivité utilisateur.

"------"
# définir les valeurs par défaut des axes x et y 
x = 'sepal length (cm)'
y = 'sepal width (cm)'# créer des sous-ensembles df par espèce pour les utiliser dans les traces
df_setosa = df[df.species == 'setosa']
df_versicolor = df[df.species == 'versicolor']
df_virginica = df[df.species == 'virginica']# créer des traces pour chaque espèce
setosa = go.Scatter(x=df_setosa[x], y=df_setosa[y],
 mode='markers',
 name='setosa')
versicolor = go.Scatter(x=df_versicolor[x], y=df_versicolor[y],
 mode='markers',
 name='versicolor')
virginica = go.Scatter(x=df_virginica[x], y=df_virginica[y],
 mode='markers',
 name='virginica')# créer l'objet graphique en tant que FigureWidget
g = go.FigureWidget(data=[setosa, versicolor, virginica],
 layout=go.Layout(
 title={
 'text': "Iris Measurements",
 'y':0.85,
 'x':0.5,
 'xanchor': 'center',
 'yanchor': 'top'},
 xaxis_title=x,
 yaxis_title=y,
 legend_title = "Iris Species"
 )) 


In [ ]:
#Maintenant, on veut que les utilisateurs puissent modifier les 
#variables sur les axes x et y. Pour ce faire, on peut créer des 
#widgets de menu déroulant pour chaque axe.

select_x = Dropdown(options = data.feature_names,
 layout = Layout(width = '160px'))
select_y = Dropdown(options = data.feature_names[::-1],
 layout = Layout(width = '160px'))
#HBox([select_x, select_y]) # pour voir un aperçu des widgets

In [ ]:
#La prochaine étape consiste à définir l'interaction entre le widget 
#et la visualisation. Écrivez une fonction pour mettre à jour les 
#valeurs des axes x et y ainsi que les étiquettes des axes. 
#Utilisez la variable "g" pour accéder à la figure.


def response(change): 
    with g.batch_update(): 
      g.data[0].x = df_setosa[select_x.value]
      g.data[1].x = df_versicolor[select_x.value]
      g.data[2].x = df_virginica[select_x.value]
      g.data[0].y = df_setosa[select_y.value]
      g.data[1].y = df_versicolor[select_y.value]
      g.data[2].y = df_virginica[select_y.value]  
      g.layout = go.Layout(
      title={
       'text': "Iris Measurements",
       'y':0.85,
       'x':0.5,
       'xanchor': 'center',
       'yanchor': 'top'},
       xaxis_title=select_x.value,
       yaxis_title=select_y.value,
       legend_title = "Iris Species"
       )

In [ ]:
#lier les widgets à la fonction de réponse lorsqu'ils sont modifiés.
#Cela se fait en utilisant la fonction observe d'IPYWidgets.

select_x.observe(response, names="value")
select_y.observe(response, names = "value")

In [ ]:
#afficher les widgets avec la visualisation.
container = HBox([select_x, select_y])
VBox([container, g])

## Les applications Voilà peuvent être déployées à l'aide de Binder, Heroku, Google App Engine ou un serveur privé. Voir [ici](https://voila.readthedocs.io/en/stable/deploy.html) pour plus d'informations.

In [ ]:
#Utilisez la commande suivante dans un terminal, à partir du 
#répertoire où se trouve ce carnet. 

#voila 4_Voila_Dashboard.ipynb